In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [13]:
#Tools Conv3D, MaxPool3D, relu, flatten, softmax
# in_channels = 3, 128x128
#will reside from 3, x, 640x480 in dataloader
# 3, 60, 224, 224
#400 total probabilities? -> even number

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv3d(3, 4, 3, padding = 'same')
        self.conv2 = nn.Conv3d(4, 8, 3, padding = 'same')
        self.conv3 = nn.Conv3d(8, 16, 3, padding = 'same')
        self.conv4 = nn.Conv3d(16, 32, 3, padding = 'same')
        self.conv5 = nn.Conv3d(32, 64, 3, padding = 'same') 

        self.conv6 = nn.Conv3d(64, 64, 3, padding = 'same')
        self.conv7 = nn.Conv3d(64, 64, 3, padding = 'same')

        self.conv8 = nn.Conv3d(64, 128, 3, padding = 'same')
        self.conv9 = nn.Conv3d(128, 128, 3, padding = 'same')
        self.conv10 = nn.Conv3d(128, 128, 3, padding = 'same')


        self.pool = nn.MaxPool3d((2, 1, 1))
        self.pool1 = nn.MaxPool3d((1, 2, 2))
        self.pool2 = nn.MaxPool3d(2)
        self.relu = nn.ReLU()

        self.predictions = nn.Linear(16*67*14*14, 400)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.relu(self.conv3(x))
        x = F.relu(self.conv4(x))
        x = self.pool1(F.relu(self.conv5(x))) # C T H W = 64 67 112 112
        temp = x
        x = F.relu(self.conv6(x))
        x = F.relu(self.conv7(x))
        x = x + temp 
        x = self.pool1(x)
        temp = x # C T H W = 64 67 56 56
        x = self.conv8(x)
        x = F.relu(self.conv9(x))
        x = self.pool(F.relu(self.conv10(x))) # error here, trying to halve number of channels through pool
        x = temp + x # C T H W = 64 67 56 56?
        x = self.pool2(x)
        x = self.pool2(x) # C T H W = 16 67 14 14
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = self.predictions(x)
        return x



In [14]:
x = torch.rand(1, 3, 67, 224, 224)

In [15]:
model = Net()

In [16]:
model(x)

RuntimeError: ignored